In [3]:
#임포트
import os,requests
from multiprocessing.managers import Value
from urllib.parse import urlparse

In [4]:
#SB_KEY = os.getenv('SAFE_BROWSING_KEY')
#쥬피터용 직접key 삽입
SB_KEY = ''
ENDPOINT = f'https://safebrowsing.googleapis.com/v4/threatMatches:find?key={SB_KEY}'



In [5]:
#4 url 정규화 (스킴 붙이기)

def _normalize_url(u:str) -> str:
    u = (u or "").strip()
    return u if "://" in u else "https://" + u

#url 정규화 과정
#만약에 프록스에 스키마가 없는 형태로 들어올수가 있음 https 같은거
#맨처음에 strip()로 빈 공백을 제거 str만 추출
#해당 url에서 스키마가 있는지 확인 :// 없으면 https:// 붙여서 u 반환 

#그런데 스킴 있느냐 없느냐 할떄" ":// 이걸로 검사



In [6]:
# url 형식 가벼운 검사

def _must_have_host(u:str) -> None:
    
    if not urlparse(u).hostname: #urlparse(u) 를 통해 hostname 유효성 검사  https:// 스키마 이후에 것들
        raise ValueError("호스트 명이 없음")
    
    

In [9]:
def _build_body(url:str)->dict: #핵심 이거 
    return {
        "client": {"clientId":"phising쉴드","clientVersion":"1.0"},
        "threatInfo":{ #병렬info
            "threatTypes":["MALWARE","SOCIAL_ENGINEERING"], #위협 분류
            "platformTypes":["ANY_PLATFORM"], # 플랫폼 설정 모든 플랫폼
            "threatEntryTypes":["URL"], #앤트리 유형 URL만
            "threatEntries":[{"url":url}] #앤트리 목록 URL
        }
    }


In [10]:
#http 호출(핵심)

def _post_browsing(body:dict)->dict:
    if not SB_KEY : raise RuntimeError("api key가 없음")
    
    res= requests.post(ENDPOINT,json=body,timeout=5) #근데 post할 데이터가 dict형식이란걸 어떻게 알음?
    res.raise_for_status(); return res.json()  #raise_for_status() -> HTTP 오류코드 4xx,5xx를 명시적으로 예외로 바꿈

In [15]:
# 단일 URL 판정(bool)
#safe Browsing 응답 규칙 : 일치하는 위협이 있으면 matches 필드가 등장
def sb_check(url:str) -> bool: 
    _normalize_url(url)
    _must_have_host(url)
    body = _build_body(url) #빈깡통 
    
    data = _post_browsing(body)
    
    return "matches" in data #data 안에 matches가 있는지 있으면 위협 상태 true,false 반환

In [16]:
bad = "http://testsafebrowsing.appspot.com/s/phishing.html"
print(sb_check(bad))

True


In [17]:
#상세 정보가 필요할떄(dict)

def sb_check_detail(url:str) -> dict:
    u = _normalize_url(url)
    _must_have_host(u)
    body = _build_body(u)
    data = _post_browsing(body)
    
    return {"url": u, "malicious": "matches" in data, "details": data.get("matches", [])}

    

In [18]:
print(sb_check_detail("http://testsafebrowsing.appspot.com/s/phishing.html"))



{'url': 'http://testsafebrowsing.appspot.com/s/phishing.html', 'malicious': True, 'details': [{'threatType': 'SOCIAL_ENGINEERING', 'platformType': 'ANY_PLATFORM', 'threat': {'url': 'http://testsafebrowsing.appspot.com/s/phishing.html'}, 'cacheDuration': '300s', 'threatEntryType': 'URL'}]}
